# Math Question Answer Verification Competition

**Goal**: Fine-tune Llama-3-8B model to predict if a given solution to a math problem is correct or not.

**Optimizations Applied**:
- Full 1M training dataset with stratified split (99.5/0.5) - 995K train, 5K validation
- LoRA rank 32 (instead of 1) for better capacity
- Max sequence length 2048 (instead of 1024) to prevent truncation
- 2 epochs training without validation during training (for maximum speed)
- Improved prompt template emphasizing solution reasoning
- Constrained decoding for reliable output parsing

**Note**: This notebook is optimized for Google Colab with A100 GPU

## **Step 1: Install Necessary Libraries**

First, we need to install the required Python libraries. We'll be using the unsloth library, which provides highly efficient, memory-saving training methods for large language models, making it possible to fine-tune powerful models on a single free-tier GPU. We'll also install xformers for further optimization.


In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9.]{3,}", torch.__version__).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-_ffq9kqj/unsloth_01293ccc84294b4391e1905f721fe24a
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-_ffq9kqj/unsloth_01293ccc84294b4391e1905f721fe24a
  Resolved https://github.com/unslothai/unsloth.git to commit f266b90718ff6e1d117c9707da9afac626a81e9d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 128.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.5 MB/s eta 0:00

## **Step 2: Hugging Face Login**

For accessing gated models like Llama-3-8B, we need to authenticate with Hugging Face using your token.



In [ ]:
from huggingface_hub import login

# Login to Hugging Face using token from Colab Secrets
# Set HF_TOKEN in Colab Secrets (Secrets → Add Secret)
try:
    from google.colab import userdata
    hf_token = userdata.get('HF_TOKEN')
    login(token=hf_token, add_to_git_credential=False)
    print("✅ Successfully logged in to Hugging Face")
except:
    print("⚠️ Warning: HF_TOKEN not found. Make sure to set it in Colab Secrets if the model is gated.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

## **Step 3: Load the Model and Tokenizer**

Load Llama-3-8B using Unsloth's FastLanguageModel with optimized settings:
- **4-bit quantization**: Reduces GPU memory usage significantly
- **Max sequence length 2048**: Prevents truncation of long problems
- **Auto dtype detection**: Optimizes for your GPU



In [ ]:
from unsloth import FastLanguageModel
import torch

# Optimized settings
max_seq_length = 2048  # Increased from 1024 to prevent truncation
dtype = None  # Auto-detect best data type for GPU
load_in_4bit = True  # Use 4-bit quantization to save memory

# Load the model and tokenizer from Hugging Face
# Using Meta-Llama-3.1-8B
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print(f"✅ Model loaded with max_seq_length={max_seq_length}")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

## **Step 4: Prepare the Dataset**

This step prepares the training data for fine-tuning. It consists of three parts:

1. **Load Dataset**: Load the full 1M training samples from Hugging Face
2. **Split Dataset**: Create train/validation split using stratified sampling (maintains class balance)
3. **Format Prompts**: Convert data into instructional prompts with improved structure

---

### 4.1 Load and Split Dataset

**Why Stratified Split?**
- Ensures training and validation sets have the same True/False ratio
- Prevents bias in validation metrics
- Better representation of the overall dataset distribution

---

### 4.2 Format Training Prompts

**Key Improvements Over Baseline:**

1. **Includes Student Answer**: The baseline prompt only had Question and Solution, but the task requires comparing the student's answer with the correct solution. This is critical!
2. **Emphasizes Reasoning**: The prompt explicitly asks the model to analyze step-by-step reasoning from the solution
3. **Clear Task Definition**: Better structure helps the model understand what it needs to do

In [ ]:
from datasets import load_dataset

# Load the full training dataset (1M samples)
full_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

# Encode is_correct as ClassLabel for stratified split
# This ensures the train/val split maintains the same True/False ratio
full_dataset = full_dataset.class_encode_column("is_correct")

# Stratified split: 99.5% training, 0.5% validation (maintains class balance)
# Using smaller validation set for faster training
split_dataset = full_dataset.train_test_split(
    test_size=0.005,  # 0.5% for validation (5,000 samples)
    seed=42,
    stratify_by_column="is_correct"
)

train_dataset = split_dataset["train"]  # ~995,000 samples
validation_dataset = split_dataset["test"]  # ~5,000 samples

print(f"✅ Training samples: {len(train_dataset):,}")
print(f"✅ Validation samples: {len(validation_dataset):,}")


In [ ]:
# Improved prompt template emphasizing solution reasoning
training_prompt = """You are an expert mathematician verifying student answers.

Your task: Determine if the student's answer is correct by analyzing the problem, the provided solution's reasoning steps, and the student's answer.

Question:
{}

Correct Solution (with step-by-step reasoning):
{}

Student Answer:
{}

Based on the solution's reasoning, is the student's answer correct?
Respond with:
- 'True' if the answer is correct
- 'False' if the answer is incorrect

Output:
{}"""

# EOS token to mark completion
EOS_TOKEN = tokenizer.eos_token

# Format data samples into the prompt template
def formatting_prompts_func(examples):
    """Format dataset examples into training prompts.
    
    Args:
        examples: Batch of dataset examples containing question, solution, answer, is_correct
    
    Returns:
        Dictionary with formatted text prompts
    """
    questions = examples["question"]
    solutions = examples["solution"]
    answers = examples["answer"]  # Student answers
    outputs = examples["is_correct"]
    texts = []
    for question, solution, answer, output in zip(questions, solutions, answers, outputs):
        # Format the prompt with all components and add EOS token
        text = training_prompt.format(
            question, 
            str(solution), 
            str(answer),
            str(output)
        ) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }

# Apply formatting to both training and validation datasets
formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
formatted_validation_dataset = validation_dataset.map(formatting_prompts_func, batched=True)

print(f"✅ Datasets formatted: {len(formatted_train_dataset):,} training, {len(formatted_validation_dataset):,} validation")

Unsloth 2025.10.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.



## **Step 5: Configure LoRA**

**LoRA (Low-Rank Adaptation)** allows us to efficiently fine-tune the model by training only a small number of adapter parameters instead of the full model.

**Optimized Settings**:
- **Rank 32**: Increased from 1 for better model capacity
- **Alpha 64**: Typically set to 2×rank for optimal scaling
- **Dropout 0.05**: Light regularization to prevent overfitting

In [ ]:
# Configure LoRA with optimized parameters
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,  # Increased from 1: more capacity for better performance
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 64,  # Typically 2×rank
    lora_dropout = 0.05,  # Light regularization
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)

model.print_trainable_parameters()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/5000 [00:00<?, ? examples/s]

## **Step 6: Set Up SFTTrainer**

Configure the training process with:
- **2 epochs**: Full passes through the 1M dataset (critical for high accuracy)
- **Learning rate 1e-4**: Stable training for large dataset
- **Warmup steps 500**: Gradual learning rate increase for stable start
- **No validation during training**: Disabled for maximum training speed
- **Save checkpoints at each epoch**: Keep last 2 checkpoints for model recovery

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 500,
        num_train_epochs = 2,
        learning_rate = 1e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 100,
        eval_strategy = "no",  # Disable validation for maximum speed
        save_strategy = "epoch",  # Save only at end of each epoch
        save_total_limit = 2,  # Keep only 2 checkpoints
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
        report_to = "none",
    ),
)

print("✅ Trainer configured: 1M samples, 2 epochs, LR 1e-4, no validation")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 2,621,440 of 8,032,882,688 (0.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.467100
20,1.092500
30,0.901400
40,0.779900
50,0.766000
60,0.771100


TrainOutput(global_step=60, training_loss=0.9629756848017375, metrics={'train_runtime': 632.943, 'train_samples_per_second': 0.758, 'train_steps_per_second': 0.095, 'total_flos': 7549047826907136.0, 'train_loss': 0.9629756848017375, 'epoch': 0.096})


## **Step 7: Start Training**

Train the model for 2 epochs over the full 995K dataset.

In [ ]:
# Start training
trainer.train()

#### QUESTION ####
Jason and Jeremy want to paint their wall white and agreed to split the cost of the paint. A gallon of paint costs $45 and can cover up to 400 square feet. How much will each of them contribute to the cost of the paint if their walls have a total area of 1600 square feet and will need a second coat?

#### SOLUTION ####
The question asks how much each of them will pay for the paint. So let's first find the total cost of the paint. Let's solve this problem using Python code.
<llm-code>
# area of one gallon of paint
area_per_gallon = 400
# number of gallons
gallons = 1600 / area_per_gallon
# price per gallon
price_per_gallon = 45
# number of coats
number_of_coats = 2
cost_of_paint = gallons * number_of_coats * price_per_gallon
cost_of_paint
</llm-code>
<llm-code-output>
360.0
</llm-code-output>
Each will pay \boxed{180} dollars.

#### MODEL'S PREDICTION ####
True<|end_of_text|>

#### CORRECT ANSWER ####
True


## **Step 8: Prepare for Inference**

Prepare the trained model for faster inference and test on a validation example to verify it's working correctly.


In [ ]:
# Prepare model for inference
FastLanguageModel.for_inference(model)

# Improved inference prompt (matching training prompt structure)
inference_prompt = """You are an expert mathematician verifying student answers.

Your task: Determine if the student's answer is correct by analyzing the problem, the provided solution's reasoning steps, and the student's answer.

Question:
{}

Correct Solution (with step-by-step reasoning):
{}

Student Answer:
{}

Based on the solution's reasoning, is the student's answer correct?
Respond with:
- 'True' if the answer is correct
- 'False' if the answer is incorrect

Output:
"""

# Test on a validation example
example = validation_dataset[10]
question = example["question"]
solution = example["solution"]
answer = example["answer"]

# Format the prompt
inputs = tokenizer(
    [inference_prompt.format(question, str(solution), str(answer))],
    return_tensors="pt",
    truncation=True,
    max_length=max_seq_length
).to("cuda")

# Generate prediction
outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
response = tokenizer.batch_decode(outputs)[0]

# Display results
print("#### QUESTION ####")
print(question)
print("\n#### SOLUTION ####")
print(solution[:200] + "..." if len(str(solution)) > 200 else solution)
print("\n#### STUDENT ANSWER ####")
print(answer)
print("\n#### MODEL'S PREDICTION ####")
output_part = response.split("Output:\n")[-1]
print(output_part[:50])
print("\n#### CORRECT ANSWER ####")
print(example["is_correct"])

 11%|█         | 1064/10000 [09:50<1:49:57,  1.35it/s]Unsloth: Input IDs of shape torch.Size([1, 1029]) with length 1029 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 13%|█▎        | 1274/10000 [11:44<1:24:15,  1.73it/s]Unsloth: Input IDs of shape torch.Size([1, 1197]) with length 1197 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 16%|█▌        | 1572/10000 [14:31<1:21:11,  1.73it/s]Unsloth: Input IDs of shape torch.Size([1, 1038]) with length 1038 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 23%|██▎       | 2329/10000 [21:37<1:15:35,  1.69it/s]Unsloth: Input IDs of shape torch.Size([1, 1211]) with length 1211 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 30%|███       | 305


Submission file 'submission.csv' created successfully!
You can now download this file and submit it to the Kaggle competition.


## **Step 9: Generate Submission File**

Generate predictions for all test samples and create the submission CSV file.

In [ ]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from transformers import LogitsProcessor, GenerationConfig
import torch

# Constrained Decoding: Force model to generate only "True"/"False" or "1"/"0" tokens
class AllowedTokensLogitsProcessor(LogitsProcessor):
    """Logits Processor that forces model to generate only allowed tokens"""
    def __init__(self, allowed_token_ids):
        self.allowed = set(allowed_token_ids)
    
    def __call__(self, input_ids, scores):
        # Set probability of disallowed tokens to -inf
        mask = torch.full_like(scores, float("-inf"))
        for tid in self.allowed:
            if tid < scores.shape[-1]:
                mask[..., tid] = 0.0
        return scores + mask

def get_allowed_token_ids(tokenizer):
    """Return token IDs for allowed tokens (True/False or 1/0)"""
    tokens = ["True", "False", "1", "0"]
    ids = []
    for token in tokens:
        token_id = tokenizer.convert_tokens_to_ids(token)
        if token_id != tokenizer.unk_token_id:
            ids.append(token_id)
    return ids

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# Setup constrained decoding
allowed_ids = get_allowed_token_ids(tokenizer)
logits_proc = AllowedTokensLogitsProcessor(allowed_ids)

# Optimized Generation configuration
gen_config = GenerationConfig(
    max_new_tokens=1,        # Reduced from 8 to 1 (faster and more accurate)
    do_sample=False,          # Deterministic generation
    temperature=0.0,          # Use probability distribution as-is
    top_p=1.0,
    eos_token_id=[tokenizer.eos_token_id],
    pad_token_id=tokenizer.eos_token_id,
)

# Generate predictions for all test samples
print(f"Generating predictions for {len(test_dataset):,} test samples...")
allowed_tokens = [tokenizer.convert_ids_to_tokens(tid) for tid in allowed_ids if tid != tokenizer.unk_token_id]
print(f"Using constrained decoding with allowed tokens: {allowed_tokens}")

for i, example in enumerate(tqdm(test_dataset)):
    question = example["question"]
    solution = example["solution"]
    answer = example["answer"]  # Student answer

    # Format the prompt
    prompt = inference_prompt.format(question, str(solution), str(answer))
    inputs = tokenizer(
        [prompt], 
        return_tensors="pt",
        truncation=True,
        max_length=max_seq_length
    ).to("cuda")

    # Generate prediction with constrained decoding
    outputs = model.generate(
        **inputs,
        generation_config=gen_config,
        logits_processor=[logits_proc],  # Apply constrained decoding
        use_cache=True
    )
    
    # Decode only newly generated tokens (more accurate parsing)
    new_tokens = outputs[0, inputs["input_ids"].shape[1]:]
    response_text = tokenizer.decode(new_tokens, skip_special_tokens=True)
    
    # Check first character only (0 or 1, True/False)
    ch = response_text.strip()[:1] if response_text.strip() else "0"
    
    # Parse: "1" or first letter "T" of "True" means True
    if ch.lower() in ["1", "t"]:
        prediction = True
    else:
        prediction = False
    
    predictions.append(prediction)

# Create submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save to CSV
submission.to_csv('submission.csv', index=False)

print(f"\n✅ Submission file created successfully!")
print(f"   Total predictions: {len(predictions):,}")
print(f"   True predictions: {sum(predictions):,}")
print(f"   False predictions: {len(predictions) - sum(predictions):,}")
print("\n📁 File saved as 'submission.csv'")
print("   You can now download this file and submit it to the Kaggle competition.")

Mounted at /content/drive


## **Step 10: Validate Submission File**

Validate the submission CSV file format before submitting to Kaggle.

**Checks**:
- File has correct columns (`ID`, `is_correct`)
- All values are `True` or `False`
- No duplicate IDs
- Exactly 10,000 rows


In [ ]:
# Validate submission file format
# This helps catch errors before submitting to Kaggle

import pandas as pd

submission_path = 'submission.csv'
df = pd.read_csv(submission_path)

# Display first few rows
print("First 5 rows:")
print(df.head())

# Check format
print(f"\nRows: {len(df)}")
print(f"Unique IDs: {df['ID'].nunique()}")
print(f"\nValue counts:")
print(df['is_correct'].value_counts(dropna=False))

# Validate format
assert set(df['is_correct'].unique()).issubset({True, False}), "is_correct must only contain True/False"
assert df['ID'].nunique() == len(df), "ID column must have unique values"
assert len(df) == 10000, f"Expected 10,000 rows, got {len(df)}"

print("\n✅ Format validation passed!")
print("   Ready to submit to Kaggle competition.")
